<a href="https://colab.research.google.com/github/Jumper15/pytorch-work/blob/main/self_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install torch

In [ ]:
import torch

# torch.manual_seed(1337)
# B, T, C = 4, 8, 1
# x = torch.randn(B, T, C)
# for b in range(B):
#   for t in range(T):
#     xbow = x[b, 0:t+1]
#     sum = torch.sum(xbow, dim=0)
#     print(sum)


tensor([0.1808])
tensor([0.1108])
tensor([-0.2488])
tensor([-1.1640])
tensor([-0.5383])
tensor([-0.5128])
tensor([0.4417])
tensor([0.5061])
tensor([0.3612])
tensor([1.5290])
tensor([0.1791])
tensor([-0.3310])
tensor([-0.0951])
tensor([-0.3349])
tensor([-1.2560])
tensor([0.2873])
tensor([1.3488])
tensor([1.2092])
tensor([1.4950])
tensor([2.4601])
tensor([0.4230])
tensor([0.9161])
tensor([2.4031])
tensor([2.9941])
tensor([0.1260])
tensor([-1.4367])
tensor([-2.5968])
tensor([-2.9316])
tensor([-2.4838])
tensor([-3.2855])
tensor([-1.7619])
tensor([0.7467])


In [ ]:
# torch.manual_seed(6767)
# tril = torch.tril(torch.ones(3, 3))
# m = torch.rand((2,3))
# t = m.T
# # sum = tril @ t
# sum = m @ tril
# m, tril, sum


In [ ]:
# torch.manual_seed(1337)
# B, T, C = 4, 8, 1
# x = torch.randn(B, T, C)
# tril = torch.tril(torch.ones(4, 8, 8))
# m = x[0].T @ tril[0]
# tril[0], x[0].view(-1), m

In [ ]:
# tril = torch.tril(torch.ones(3,4))
# tril = tril / torch.sum(tril, dim=1, keepdims=True)
# tril

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000]])

In [64]:
# key, query, value
# n_dims = 16
# B, T, C = 4, 8, 2

import torch
import torch.nn as nn
torch.manual_seed(676767)

B, T, C = 4, 8, 2
embed_dims = 32

# for one batch
input = torch.randint(8, (B, T, C)) # TC: 4, 8, 2
Embed = nn.Embedding(8, embed_dims) # embeds: 4, 8, 2, 32
# input, Embed(input)
embed = Embed(input)
embed = embed.view(B, T, -1)
ln1 = nn.Linear(embed_dims*C, C)
logits = ln1(embed)

input.shape, embed.shape, logits.shape


(torch.Size([4, 8, 2]), torch.Size([4, 8, 64]), torch.Size([4, 8, 2]))

In [73]:
torch.manual_seed(676767)

# attention block
head_size = 16

# value_layer = nn.Linear(C, head_size)
key_layer = nn.Linear(C, head_size)
query_layer = nn.Linear(C, head_size)

keys = key_layer(logits)
querys = query_layer(logits)
attention_block = keys @ querys.transpose(-1, -2)
attention_block.shape, attention_block[0]


(torch.Size([4, 8, 8]),
 tensor([[ 0.2850,  0.7048,  0.6008,  0.2850,  0.5184,  1.0615,  0.5184,  0.5212],
         [ 0.8059,  1.3491,  0.5115,  0.8059,  0.3369,  1.8481,  0.3369,  0.0242],
         [ 0.2421,  0.6112, -0.3186,  0.2421, -0.4721,  0.9696, -0.4721, -0.8469],
         [ 0.2850,  0.7048,  0.6008,  0.2850,  0.5184,  1.0615,  0.5184,  0.5212],
         [ 0.0979,  0.4289, -0.3835,  0.0979, -0.5192,  0.7491, -0.5192, -0.8456],
         [ 1.2717,  1.9274,  0.4811,  1.2717,  0.2283,  2.5531,  0.2283, -0.3450],
         [ 0.0979,  0.4289, -0.3835,  0.0979, -0.5192,  0.7491, -0.5192, -0.8456],
         [-0.0943,  0.1730, -0.7676, -0.0943, -0.9046,  0.4467, -0.9046, -1.2963]],
        grad_fn=<SelectBackward0>))

In [95]:
torch.manual_seed(676767)

# tril
sfmx = nn.Softmax(dim=2)
masked_tril = attention_block.masked_fill(torch.tril(torch.ones(4, 8, 8)) == 0, float('-inf'))
output = sfmx(masked_tril)
attention_block[0], masked_tril[0], output[0]


(tensor([[ 0.2850,  0.7048,  0.6008,  0.2850,  0.5184,  1.0615,  0.5184,  0.5212],
         [ 0.8059,  1.3491,  0.5115,  0.8059,  0.3369,  1.8481,  0.3369,  0.0242],
         [ 0.2421,  0.6112, -0.3186,  0.2421, -0.4721,  0.9696, -0.4721, -0.8469],
         [ 0.2850,  0.7048,  0.6008,  0.2850,  0.5184,  1.0615,  0.5184,  0.5212],
         [ 0.0979,  0.4289, -0.3835,  0.0979, -0.5192,  0.7491, -0.5192, -0.8456],
         [ 1.2717,  1.9274,  0.4811,  1.2717,  0.2283,  2.5531,  0.2283, -0.3450],
         [ 0.0979,  0.4289, -0.3835,  0.0979, -0.5192,  0.7491, -0.5192, -0.8456],
         [-0.0943,  0.1730, -0.7676, -0.0943, -0.9046,  0.4467, -0.9046, -1.2963]],
        grad_fn=<SelectBackward0>),
 tensor([[ 0.2850,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
         [ 0.8059,  1.3491,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
         [ 0.2421,  0.6112, -0.3186,    -inf,    -inf,    -inf,    -inf,    -inf],
         [ 0.2850,  0.7048,  0.6008,  0.2850,    -

In [3]:
import torch
import torch.nn as nn
torch.manual_seed(676767)

class Attention_Head():
  def __init__(self, head_size=16, embed_dims=32):
    self.head_size = head_size
    self.embed_dims = embed_dims

    self.Embed = nn.Embedding(8, embed_dims)

  def masked_tril(self, logits):
    sfmx = nn.Softmax(dim=2)
    masked_tril = logits.masked_fill(torch.tril(torch.ones(4, 8, 8)) == 0, float('-inf'))
    output = sfmx(masked_tril)
    return output

  def self_attention(self, input):
    # B, T, C = 4, 8, 2
    B, T, C = input.shape

    # for one batch
    # Embed = nn.Embedding(8, embed_dims) # embeds: 4, 8, 2, 32
    # input, Embed(input)
    embed = self.Embed(input)
    embed = embed.view(B, T, -1)
    ln1 = nn.Linear(self.embed_dims*C, C)
    logits = ln1(embed)

    # attention block
    head_size = 16

    # value_layer = nn.Linear(C, head_size)
    key_layer = nn.Linear(C, self.head_size)
    query_layer = nn.Linear(C, self.head_size)

    keys = key_layer(logits)
    querys = query_layer(logits)
    attention_block = keys @ querys.transpose(-1, -2)
    attention_block.shape, attention_block[0]

    # tril
    # sfmx = nn.Softmax(dim=2)
    # masked_tril = attention_block.masked_fill(torch.tril(torch.ones(4, 8, 8)) == 0, float('-inf'))
    # output = sfmx(masked_tril)
    # attention_block[0], masked_tril[0], output[0]



    input = torch.randint(8, (B, T, C)) # TC: 4, 8, 2


IndentationError: expected an indented block after function definition on line 12 (ipython-input-1212774688.py, line 14)